In [3]:
import requests
import pandas as pd
import numpy as np
from scipy.stats import norm
from datetime import datetime, timedelta, timezone
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")

API_KEY = "67ffece4b2ae08.94077168"
BASE_URL = "https://eodhd.com/api/mp/unicornbay/options/contracts"

# ----------------------------------------------------------------------------------------
# Black-Scholes Option Pricing Function Parameters
#
# S     : float - Current price of the underlying stock (spot price)
#         Example: If AAPL is trading at $175, then S = 175
#
# K     : float - Strike price of the option contract
#         The fixed price at which the option holder can buy (call) or sell (put) the stock
#
# T     : float - Time to expiration in years
#         Example: If the option expires in 30 days, then T = 30 / 365 ≈ 0.0822
#
# r     : float - Risk-free interest rate (annual), expressed as a decimal
#         Example: If the rate is 5%, then r = 0.05
#
# sigma : float - Volatility of the underlying stock’s returns (annualized), expressed as a decimal
#         Usually implied volatility (IV). Example: 30% volatility → sigma = 0.30
#
# Returns:
#   Theoretical price of a European call or put option using the Black-Scholes model

def black_scholes_call_price(S, K, T, r, sigma):
    if T <= 0 or sigma <= 0:
        return max(S - K, 0)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

def black_scholes_put_price(S, K, T, r, sigma):
    if T <= 0 or sigma <= 0:
        return max(K - S, 0)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

def current_stock_price(ticker):
    url = f"https://eodhd.com/api/real-time/{ticker}.US?api_token={API_KEY}&fmt=json"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        data = response.json()
        return float(data.get("close", 0))
    except Exception as e:
        logging.warning(f"Failed to fetch current price for {ticker}: {e}")
        return 0

def fetch_filtered_options(symbol, exp_from, exp_to, strike_from, strike_to, option_type):
    params = {
        "api_token": API_KEY,
        "filter[underlying_symbol]": symbol,
        "filter[type]": option_type,
        "filter[exp_date_from]": exp_from,
        "filter[exp_date_to]": exp_to,
        "filter[strike_from]": strike_from,
        "filter[strike_to]": strike_to,
        "sort": "exp_date",
        "page[limit]": 1000
    }

    try:
        response = requests.get(BASE_URL, params=params, timeout=10)
        response.raise_for_status()
        return response.json().get("data", [])
    except Exception as e:
        logging.warning(f"Error fetching options contracts for {symbol}: {e}")
        return []

def analyze_options_unicorn(
    tickers,
    option_type="call",
    days_until_exp=90,
    strike_pct=0.2,
    days_to_gain=30,
    stock_gain_pct=0.1,
    risk_free_rate=0.05,
    allocation=None
):
    today = datetime.now(timezone.utc)
    eval_date = today + timedelta(days=days_to_gain)
    exp_from = (today + timedelta(days=days_until_exp - 30)).strftime("%Y-%m-%d")
    exp_to = (today + timedelta(days=days_until_exp + 30)).strftime("%Y-%m-%d")

    results = []

    for ticker in tickers:
        try:
            current_price = current_stock_price(ticker)
            if current_price <= 0:
                logging.info(f"Skipping {ticker}: invalid current price.")
                continue

            target_strike = current_price * (1 + strike_pct)
            lower = target_strike * 0.95
            upper = target_strike * 1.05

            options_data = fetch_filtered_options(ticker, exp_from, exp_to, lower, upper, option_type)
            if not options_data:
                logging.info(f"No options data returned for {ticker}")
                continue

            # Group by expiration date and keep 2 closest strikes per group
            grouped = {}
            for opt in options_data:
                attr = opt.get("attributes", {})
                exp = attr.get("exp_date")
                if exp not in grouped:
                    grouped[exp] = []
                grouped[exp].append(opt)

            filtered_options = []
            for exp, group in grouped.items():
                closest = sorted(
                    group,
                    key=lambda x: abs(x.get("attributes", {}).get("strike", 0) - target_strike)
                )[:2]
                filtered_options.extend(closest)

            for opt in filtered_options:
                attr = opt.get("attributes", {})
                strike = attr.get("strike")
                exp_date_str = attr.get("exp_date")
                last_price = attr.get("last", 0)
                last_Premium = attr.get("last", 0)
                iv = attr.get("volatility", 0.3)

                delta = attr.get("delta")
                theta = attr.get("theta")
                gamma = attr.get("gamma") 
                vega = attr.get("vega")
                rho = attr.get("rho")

                bid = attr.get("bid")
                bid_date = attr.get("bid_date")
                ask = attr.get("ask")
                ask_date = attr.get("ask_date")
                volume = attr.get("volume")

                if not strike or not exp_date_str:
                    continue

                if not last_price and bid is not None and ask is not None:
                    try:
                        last_price = (float(bid) + float(ask)) / 2
                    except:
                        last_price = 0

                exp_date = datetime.strptime(exp_date_str, "%Y-%m-%d").replace(tzinfo=timezone.utc)
                exp_days = (exp_date - today).days
                T_eval = max((exp_date - eval_date).days / 365, 0.0001)

                sim_stock = current_price * (1 + stock_gain_pct)

                if option_type == "call":
                    est_value = black_scholes_call_price(sim_stock, strike, T_eval, risk_free_rate, iv)
                    pct_otm_itm = (strike - current_price) / current_price * 100
                elif option_type == "put":
                    est_value = black_scholes_put_price(sim_stock, strike, T_eval, risk_free_rate, iv)
                    pct_otm_itm = (current_price - strike) / current_price * 100
                else:
                    est_value = None
                    pct_otm_itm = "NA"

                pct_gain = ((est_value - last_price) / last_price) * 100 if last_price and est_value else None
                alloc_value = allocation if allocation else None
                simulated_value = (alloc_value * (1 + pct_gain / 100)) if alloc_value and pct_gain is not None else None

                result = {
                    "Ticker": ticker,
                    "Option Type": option_type,
                    "Expiration": exp_date_str,
                    "Strike": strike,
                    "% OTM/ITM": round(pct_otm_itm, 2) if isinstance(pct_otm_itm, (int, float)) else pct_otm_itm,
                    "Current Price": round(current_price, 2),
                    "Simulated Price": round(sim_stock, 2),
                    "Current Premium": round(last_price, 2),
                    "Simulated Premium": round(est_value, 2) if est_value is not None else "NA",
                    "Days Until Expiration": exp_days,
                    "Days to Gain": days_to_gain,
                    "Stock Gain %": round(stock_gain_pct * 100, 2),
                    "Premium % Gain": round(pct_gain, 2) if pct_gain is not None else "NA",
                    "Implied Volatility": round(iv * 100, 2),
                    "Delta": round(delta, 4) if delta is not None else "NA",
                    "Theta": round(theta, 4) if theta is not None else "NA",
                    "Gamma": round(gamma, 4) if gamma is not None else "NA",
                    "Vega": round(vega, 4) if vega is not None else "NA",
                    "Rho": round(rho, 4) if rho is not None else "NA",
                    "Bid": bid,
                    "Ask": ask,
                    "Last Premium": last_Premium,
                    "Bid Date": bid_date,
                    "Ask Date": ask_date,
                    "Volume": volume
                }

                if alloc_value is not None and simulated_value is not None:
                    result["Allocated ($)"] = round(alloc_value, 2)
                    result["Simulated Equity ($)"] = round(simulated_value, 2)

                results.append(result)

        except Exception as e:
            logging.error(f"Error processing {ticker}: {e}")

    return pd.DataFrame(results)

def run_multiple_analyses(param_sets):
    combined_df = pd.DataFrame()
    for params in param_sets:
        label = params.pop("label", "")
        allocation = params.pop("allocation", None)
        df = analyze_options_unicorn(**params, allocation=allocation)
        df["Run_Label"] = label
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    return combined_df


def highlightsTable(df):
    cols = [
        "Ticker",
        "Option Type",
        "Expiration",
        "Strike",
        "Current Stock",
        "Current Premium",
        "Stock Gain %",
        "Premium % Gain",
        "Implied Volatility",
        "Bid Date",
        "Volume"
    ]
    return df[cols]

# # Example use
# param_sets = [
#     {
#         "tickers": ["AAPL"],
#         "option_type": "call",
#         "days_until_exp": 120,
#         "strike_pct": 0.2,
#         "days_to_gain": 30,
#         "stock_gain_pct": 0.1,
#         "allocation": 100,
#         "label": "Short-Term Conservative"
#     },
#     {
#         "tickers": ["AAPL"],
#         "option_type": "put",
#         "days_until_exp": 120,
#         "strike_pct": -0.2,
#         "days_to_gain": 30,
#         "stock_gain_pct": -0.1,
#         "allocation": 40,
#         "label": "Long-Term Aggressive (Put)"
#     }
# ]

# combined_results = run_multiple_analyses(param_sets)
# display(combined_results)
# # combined_results.to_csv("optionsTests.csv", index=False)


,Ticker,Option Type,Expiration,Strike,% OTM/ITM,Current Stock,Simulated Stock,Current Premium,Simulated Premium,Days Until Expiration,...,Theta,Bid,Ask,Last Premium,Bid Date,Ask Date,Volume,Allocated ($),Simulated Value ($),Run_Label
0,AAPL,call,2025-09-19,240,20.88,198.54,218.39,1.64,3.77,98,...,-0.0316,1.64,1.68,1.64,2025-06-11 19:59:59,2025-06-11 19:59:59,1960,100,229.89,Short-Term Conservative
1,AAPL,call,2025-09-19,235,18.36,198.54,218.39,2.18,4.99,98,...,-0.0372,2.17,2.22,2.18,2025-06-11 19:59:59,2025-06-11 19:59:59,368,100,229.12,Short-Term Conservative
2,AAPL,call,2025-10-17,240,20.88,198.54,218.39,2.43,5.46,126,...,-0.0328,2.41,2.45,2.43,2025-06-11 19:59:59,2025-06-11 19:59:59,201,100,224.65,Short-Term Conservative
3,AAPL,call,2025-10-17,235,18.36,198.54,218.39,3.12,6.88,126,...,-0.0376,3.05,3.15,3.12,2025-06-11 19:59:59,2025-06-11 19:59:59,84,100,220.60,Short-Term Conservative
4,AAPL,put,2025-09-19,160,19.41,198.54,178.68,2.10,3.60,98,...,-0.0329,2.05,2.12,2.10,2025-06-11 19:59:59,2025-06-11 19:59:59,177,40,68.49,Long-Term Aggressive (Put)
5,AAPL,put,2025-09-19,155,21.93,198.54,178.68,1.65,2.70,98,...,-0.0288,1.65,1.69,1.65,2025-06-11 19:59:59,2025-06-11 19:59:59,19,40,65.40,Long-Term Aggressive (Put)
6,AAPL,put,2025-10-17,160,19.41,198.54,178.68,2.77,4.74,126,...,-0.0308,2.72,2.77,2.77,2025-06-11 19:59:59,2025-06-11 19:59:59,28,40,68.39,Long-Term Aggressive (Put)
7,AAPL,put,2025-10-17,155,21.93,198.54,178.68,2.24,3.73,126,...,-0.0276,2.21,2.25,2.24,2025-06-11 19:59:59,2025-06-11 19:59:59,5,40,66.62,Long-Term Aggressive (Put)


2025-06-12 11:06:43,152 [WARNING] No contract found for AAPL call 185.0 2025-12-20


Empty DataFrame
Columns: []
Index: []


In [2]:
import requests
import json

# API_KEY = "67ffece4b2ae08.94077168"
BASE_URL = "https://eodhd.com/api/mp/unicornbay/options/contracts"

def fetch_specific_option(symbol, strike, exp_date, option_type="call"):
    params = {
        "api_token": API_KEY,
        "filter[underlying_symbol]": symbol,
        "filter[strike_eq]": strike,
        "filter[exp_date_eq]": exp_date,
        "filter[type]": option_type,
        "page[limit]": 1
    }

    try:
        response = requests.get(BASE_URL, params=params, timeout=10)
        response.raise_for_status()
        data = response.json().get("data", [])
        if not data:
            print(f"No option found for {symbol} {strike} {option_type.upper()} expiring {exp_date}")
            return

        contract = data[0].get("attributes", {})
        print(json.dumps(contract, indent=4))  # Pretty-print the full contract data

    except Exception as e:
        print(f"Error fetching option: {e}")

# Example usage
if __name__ == "__main__":
    fetch_specific_option(
        symbol="AAPL",
        strike=225,
        exp_date="2025-07-18",
        option_type="call"
    )


{
    "contract": "AAPL250718C00225000",
    "underlying_symbol": "AAPL",
    "exp_date": "2025-07-18",
    "expiration_type": "monthly",
    "type": "call",
    "strike": 225,
    "exchange": "NASDAQ",
    "currency": "USD",
    "open": 1.02,
    "high": 1.21,
    "low": 0.84,
    "last": 1.01,
    "last_size": 1,
    "change": -0.08,
    "pctchange": -7.34,
    "previous": 1.09,
    "previous_date": "2025-06-09",
    "bid": 1.01,
    "bid_date": "2025-06-10 20:00:00",
    "bid_size": 26,
    "ask": 1.04,
    "ask_date": "2025-06-10 20:00:00",
    "ask_size": 55,
    "moneyness": -0.10999999999999999,
    "volume": 2919,
    "volume_change": -3598,
    "volume_pctchange": -55.21,
    "open_interest": 24013,
    "open_interest_change": 760,
    "open_interest_pctchange": 3.27,
    "volatility": 0.2604,
    "volatility_change": -0.0122,
    "volatility_pctchange": -4.46,
    "theoretical": 1.01,
    "delta": 0.123965,
    "gamma": 0.012014,
    "theta": -0.048339,
    "vega": 0.133818,


In [21]:
def simulate_option_watchlist_single(ticker, option_type, strike, expiration, days_to_gain,
                                     number_of_contracts, average_cost_per_contract, risk_free_rate=0.05):
    try:
        current_price = current_stock_price(ticker)
        if current_price <= 0:
            logging.warning("Invalid stock price for %s", ticker)
            return pd.DataFrame()

        # Fetch exact contract data from EODHD
        params = {
            "api_token": API_KEY,
            "filter[underlying_symbol]": ticker,
            "filter[type]": option_type,
            "filter[exp_date_from]": expiration,
            "filter[exp_date_to]": expiration,
            "filter[strike_from]": strike,
            "filter[strike_to]": strike,
            "page[limit]": 1
        }
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()
        data = response.json().get("data", [])

        if not data or "attributes" not in data[0]:
            logging.warning(f"No valid contract found for {ticker} {option_type} {strike} {expiration}")
            return pd.DataFrame()

        attr = data[0]["attributes"]
        last_price = float(attr.get("last", 0))
        iv = float(attr.get("volatility", 0.3))

        if last_price <= 0:
            logging.warning(f"Missing or invalid last premium for {ticker}")
            return pd.DataFrame()

        exp_date = datetime.strptime(expiration, "%Y-%m-%d").replace(tzinfo=timezone.utc)
        eval_date = datetime.now(timezone.utc) + timedelta(days=days_to_gain)
        T_eval = max((exp_date - eval_date).days / 365, 0.0001)

        total_cost = average_cost_per_contract * number_of_contracts * 100
        current_gain_pct = ((last_price - average_cost_per_contract) / average_cost_per_contract) * 100
        current_gain_dollar = (last_price - average_cost_per_contract) * number_of_contracts * 100

        # Define the % stock movement scenarios
        scenarios = [0.05, 0.10, 0.20, 0.50]
        rows = []

        for pct in scenarios:
            stock_up = current_price * (1 + pct)
            stock_down = current_price * (1 - pct)

            if option_type == "call":
                premium_up = black_scholes_call_price(stock_up, strike, T_eval, risk_free_rate, iv)
                premium_down = black_scholes_call_price(stock_down, strike, T_eval, risk_free_rate, iv)
            elif option_type == "put":
                premium_up = black_scholes_put_price(stock_up, strike, T_eval, risk_free_rate, iv)
                premium_down = black_scholes_put_price(stock_down, strike, T_eval, risk_free_rate, iv)
            else:
                continue

            pct_up_gain = ((premium_up - last_price) / last_price) * 100
            pct_down_gain = ((premium_down - last_price) / last_price) * 100
            
            rows.append({
                # Contract Info
                "Ticker": ticker,
                "Option Type": option_type,
                "Strike": strike,
                "Expiration": expiration,
                "Implied Volatility": round(iv * 100, 2),
            
                # Position Details
                "Number of Contracts": number_of_contracts,
                "Average Cost per Contract": average_cost_per_contract,
                "Average Cost ($)": round(total_cost, 2),
                "Current Premium": round(last_price, 2),
                "Your Equity ($)": round(last_price * number_of_contracts * 100, 2),
                "Total % Change": round(current_gain_pct, 2),
            
                # Scenario Setup
                "Days to Gain": days_to_gain,
                "Scenario %": f"±{int(pct * 100)}%",
                "Stock Price Up": round(stock_up, 2),
                "Stock Price Down": round(stock_down, 2),
            
                # Simulated Premiums
                "Premium Up": round(premium_up, 2),
                "Premium Down": round(premium_down, 2),
            
                # Simulated Returns
                "% Gain from Current (Up)": round(pct_up_gain, 2),
                "% Gain from Current (Down)": round(pct_down_gain, 2)
            })

        return pd.DataFrame(rows)

    except Exception as e:
        logging.error(f"Error simulating option scenario for {ticker}: {e}")
        return pd.DataFrame()


def whole_watchlist(contract_list):
    all_rows = []

    for contract in contract_list:
        df = simulate_option_watchlist_single(
            ticker=contract["ticker"],
            option_type=contract["option_type"],
            strike=contract["strike"],
            expiration=contract["expiration"],
            days_to_gain=contract["days_to_gain"],
            number_of_contracts=contract["number_of_contracts"],
            average_cost_per_contract=contract["average_cost_per_contract"]
        )

        if not df.empty:
            all_rows.append(df)

    return pd.concat(all_rows, ignore_index=True) if all_rows else pd.DataFrame()



In [18]:
# df = simulate_option_watchlist_single(
#     ticker="AAPL",
#     option_type="call",
#     strike=220.0,
#     expiration="2025-12-19",
#     days_to_gain=30,
#     number_of_contracts=3,
#     average_cost_per_contract=4.5
# )

# display(df)
# # df.to_csv("watchlist_scenarios.csv", index=False)


,Ticker,Option Type,Strike,Expiration,Days to Gain,Scenario %,Stock Price Up,Stock Price Down,Premium Up,Premium Down,% Gain from Current (Up),% Gain from Current (Down),Current Premium,Implied Volatility,Number of Contracts,Average Cost per Contract,Average Cost ($),Total % Change,Your Equity ($)
0,AAPL,call,220.0,2025-12-19,30,±5%,207.95,188.14,12.34,5.16,28.54,-46.27,9.6,28.27,3,4.5,1350.0,113.33,2880.0
1,AAPL,call,220.0,2025-12-19,30,±10%,217.85,178.24,17.42,2.97,81.50,-69.05,9.6,28.27,3,4.5,1350.0,113.33,2880.0
2,AAPL,call,220.0,2025-12-19,30,±20%,237.65,158.44,30.33,0.72,215.98,-92.49,9.6,28.27,3,4.5,1350.0,113.33,2880.0
3,AAPL,call,220.0,2025-12-19,30,±50%,297.07,99.02,82.61,0.00,760.56,-100.00,9.6,28.27,3,4.5,1350.0,113.33,2880.0


In [24]:
# # bought around march 26, 2025

# watchlist = [
#     {
#         "ticker": "MSFT",
#         "option_type": "call",
#         "strike": 500.0,
#         "expiration": "2025-12-19",
#         "days_to_gain": 30,
#         "number_of_contracts": 3,
#         "average_cost_per_contract": 5.85
#     },
#     {
#         "ticker": "AAPL",
#         "option_type": "call",
#         "strike": 220.0,
#         "expiration": "2025-12-19",
#         "days_to_gain": 30,
#         "number_of_contracts": 3,
#         "average_cost_per_contract": 25.23
#     }
# ]

# watchlist_df = whole_watchlist(watchlist)
# display(watchlist_df)
# watchlist_df.to_csv("watchlist_output.csv", index=False)


,Ticker,Option Type,Strike,Expiration,Implied Volatility,Number of Contracts,Average Cost per Contract,Average Cost ($),Current Premium,Your Equity ($),Total % Change,Days to Gain,Scenario %,Stock Price Up,Stock Price Down,Premium Up,Premium Down,% Gain from Current (Up),% Gain from Current (Down)
0,MSFT,call,500.0,2025-12-19,22.11,3,5.85,1755.0,22.45,6735.0,283.76,30,±5%,500.57,452.89,34.81,12.97,55.05,-42.24
1,MSFT,call,500.0,2025-12-19,22.11,3,5.85,1755.0,22.45,6735.0,283.76,30,±10%,524.40,429.06,50.33,6.71,124.21,-70.10
2,MSFT,call,500.0,2025-12-19,22.11,3,5.85,1755.0,22.45,6735.0,283.76,30,±20%,572.08,381.38,88.44,1.13,293.92,-94.96
3,MSFT,call,500.0,2025-12-19,22.11,3,5.85,1755.0,22.45,6735.0,283.76,30,±50%,715.10,238.37,225.99,0.00,906.65,-100.00
4,AAPL,call,220.0,2025-12-19,28.27,3,25.23,7569.0,9.60,2880.0,-61.95,30,±5%,207.76,187.98,12.26,5.11,27.66,-46.72
5,AAPL,call,220.0,2025-12-19,28.27,3,25.23,7569.0,9.60,2880.0,-61.95,30,±10%,217.66,178.08,17.32,2.94,80.37,-69.34
6,AAPL,call,220.0,2025-12-19,28.27,3,25.23,7569.0,9.60,2880.0,-61.95,30,±20%,237.44,158.30,30.18,0.71,214.38,-92.57
7,AAPL,call,220.0,2025-12-19,28.27,3,25.23,7569.0,9.60,2880.0,-61.95,30,±50%,296.81,98.94,82.36,0.00,757.92,-100.00
